In [1]:
import neo4j

import csv

import math
import numpy as np
import pandas as pd

import psycopg2

In [2]:
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","ucb_mids_w205"))

In [3]:
session = driver.session(database="neo4j")

In [4]:
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

In [5]:
def my_neo4j_wipe_out_mst_relationships():
    "wipe out mst relationships"
    
    query = "match (node)-[relationship:MST]->() delete relationship"
    session.run(query)

In [23]:
my_neo4j_wipe_out_mst_relationships()

In [20]:
query = "CALL gds.graph.drop('ds_graph', false)"
session.run(query)

query = """

CALL gds.graph.project('ds_graph', 'Station', 
                        {
                            LINK: {
                                properties: 'weight',
                                orientation: 'UNDIRECTED'
                            }
                        }
                       )

"""

session.run(query)

In [21]:
query = """

MATCH (n:Station {name: $source})
CALL gds.beta.spanningTree.write('ds_graph',
                                          {sourceNode: id(n),
                                           relationshipWeightProperty: 'weight',
                                           writeProperty: 'writeCost',
                                           writeRelationshipType: 'MST'
                                          }
                                         )
YIELD preProcessingMillis, computeMillis, writeMillis, effectiveNodeCount
RETURN preProcessingMillis, computeMillis, writeMillis, effectiveNodeCount;

"""

source = "Dublin"

my_neo4j_run_query_pandas(query, source=source)

,preProcessingMillis,computeMillis,writeMillis,effectiveNodeCount


In [22]:
query = """

MATCH path = (n:Station {name: $source})-[:MST*]-()
WITH relationships(path) AS rels
UNWIND rels AS rel
WITH DISTINCT rel AS rel
RETURN startNode(rel).name AS source, endNode(rel).name AS destination, rel.writeCost AS cost

"""

source = "Dublin"

my_neo4j_run_query_pandas(query, source=source)

,source,destination,cost


In [24]:
query = "CALL gds.graph.drop('ds_graph', false)"
session.run(query)

query = """

CALL gds.graph.project('ds_graph', 'Station', 'LINK', 
                      {relationshipProperties: 'weight'})
"""

session.run(query)

In [27]:
query = """

CALL gds.louvain.stream('ds_graph', {includeIntermediateCommunities: true})
YIELD nodeId, communityId, intermediateCommunityIds
RETURN gds.util.asNode(nodeId).name AS name, communityId as community, intermediateCommunityIds as intermediate_community
ORDER BY community, name ASC

"""

my_neo4j_run_query_pandas(query)


,name,community,intermediate_community
0,arrive 12th Street,39,"[39, 39, 39]"
1,arrive 19th Street,39,"[43, 39, 39]"
2,arrive MacArthur,39,"[91, 39, 39]"
3,depart 12th Street,39,"[39, 39, 39]"
4,depart 19th Street,39,"[43, 39, 39]"
...,...,...,...
209,yellow Pittsburg,131,"[107, 47, 131]"
210,yellow Pittsburg Center,131,"[109, 47, 131]"
211,yellow Pleasant Hill,131,"[111, 131, 131]"
212,yellow Rockridge,131,"[117, 117, 131]"


In [30]:
query = "CALL gds.graph.drop('ds_graph', false)"
session.run(query)

query = "CALL gds.graph.project('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})"
session.run(query)

In [31]:
query = """

CALL gds.alpha.allShortestPaths.stream('ds_graph', 
                                       {relationshipWeightProperty: 'weight'}
                                      )
YIELD sourceNodeId, targetNodeId, distance
WITH sourceNodeId, targetNodeId, distance
WHERE gds.util.isFinite(distance) = true

MATCH (source:Station) WHERE id(source) = sourceNodeId
MATCH (target:Station) WHERE id(target) = targetNodeId
WITH source, target, distance WHERE source <> target

RETURN source.name AS source, target.name AS target, distance
ORDER BY distance DESC, source ASC, target ASC

"""

my_neo4j_run_query_pandas(query)

,source,target,distance
0,depart Antioch,green Berryessa,7073.0
1,green Berryessa,arrive Antioch,7073.0
2,green Berryessa,yellow Antioch,7073.0
3,yellow Antioch,green Berryessa,7073.0
4,depart Antioch,arrive Berryessa,7019.0
...,...,...,...
26777,yellow SFO,arrive SFO,0.0
26778,yellow San Bruno,arrive San Bruno,0.0
26779,yellow South San Francisco,arrive South San Francisco,0.0
26780,yellow Walnut Creek,arrive Walnut Creek,0.0


In [32]:
def my_get_node_list():
    "get a list of nodes in the current graph"
    
    query = "match (n) return n.name as name"
    
    result = session.run(query)
    
    node_list = []
    
    for r in result:
        node_list.append(r["name"])
        
    node_list = sorted(node_list)
    
    return node_list

In [33]:
query = "CALL gds.graph.drop('ds_graph', false)"
session.run(query)

query = "CALL gds.graph.project('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})"
session.run(query)

In [34]:
query = """

MATCH (source:Station {name: $source}), (target:Station {name: $target})
CALL gds.shortestPath.dijkstra.stream(
    'ds_graph', 
    { sourceNode: source, 
      targetNode: target, 
      relationshipWeightProperty: 'weight'
    }
)
YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
RETURN
    gds.util.asNode(sourceNode).name AS from,
    gds.util.asNode(targetNode).name AS to,
    totalCost,
    [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodes,
    costs
ORDER BY index

"""

nodes = my_get_node_list()

all_pairs_shortest_paths = []

for source in nodes:
    
    for target in nodes:
        
        if source != target:
            
            result = session.run(query, source=source, target=target)
            
            for r in result:

                all_pairs_shortest_paths.append([r["from"], r["to"], r["totalCost"], r["nodes"], r["costs"]])
                
for path in all_pairs_shortest_paths:
    
    print(path)
                    

['blue 16th Street Mission', 'arrive 12th Street', 1423.0, ['blue 16th Street Mission', 'blue Civic Center', 'blue Powell Street', 'blue Montgomery Street', 'blue Embarcadero', 'blue West Oakland', 'yellow West Oakland', 'yellow 12th Street', 'arrive 12th Street'], [0.0, 180.0, 240.0, 360.0, 420.0, 840.0, 1123.0, 1423.0, 1423.0]]
['blue 16th Street Mission', 'arrive 16th Street Mission', 0.0, ['blue 16th Street Mission', 'arrive 16th Street Mission'], [0.0, 0.0]]
['blue 16th Street Mission', 'arrive 19th Street', 1543.0, ['blue 16th Street Mission', 'blue Civic Center', 'blue Powell Street', 'blue Montgomery Street', 'blue Embarcadero', 'blue West Oakland', 'red West Oakland', 'red 12th Street', 'red 19th Street', 'arrive 19th Street'], [0.0, 180.0, 240.0, 360.0, 420.0, 840.0, 1123.0, 1423.0, 1543.0, 1543.0]]
['blue 16th Street Mission', 'arrive 24th Street Mission', 120.0, ['blue 16th Street Mission', 'blue 24th Street Mission', 'arrive 24th Street Mission'], [0.0, 120.0, 120.0]]
['bl

['blue Fruitvale', 'arrive Glen Park', 1800.0, ['blue Fruitvale', 'blue Lake Merritt', 'blue West Oakland', 'blue Embarcadero', 'blue Montgomery Street', 'blue Powell Street', 'blue Civic Center', 'blue 16th Street Mission', 'blue 24th Street Mission', 'blue Glen Park', 'arrive Glen Park'], [0.0, 300.0, 660.0, 1080.0, 1140.0, 1260.0, 1320.0, 1500.0, 1620.0, 1800.0, 1800.0]]
['blue Fruitvale', 'arrive Hayward', 1014.0, ['blue Fruitvale', 'blue Coliseum', 'orange Coliseum', 'orange San Leandro', 'orange Bay Fair', 'orange Hayward', 'arrive Hayward'], [0.0, 240.0, 294.0, 534.0, 774.0, 1014.0, 1014.0]]
['blue Fruitvale', 'arrive Lafayette', 1958.0, ['blue Fruitvale', 'orange Fruitvale', 'orange Lake Merritt', 'orange 12th Street', 'orange 19th Street', 'orange MacArthur', 'yellow MacArthur', 'yellow Rockridge', 'yellow Orinda', 'yellow Lafayette', 'arrive Lafayette'], [0.0, 279.0, 579.0, 759.0, 879.0, 1059.0, 1118.0, 1358.0, 1658.0, 1958.0, 1958.0]]
['blue Fruitvale', 'arrive Lake Merritt'

['depart Ashby', 'arrive Glen Park', 1980.0, ['depart Ashby', 'red Ashby', 'red MacArthur', 'red 19th Street', 'red 12th Street', 'red West Oakland', 'red Embarcadero', 'red Montgomery Street', 'red Powell Street', 'red Civic Center', 'red 16th Street Mission', 'red 24th Street Mission', 'red Glen Park', 'arrive Glen Park'], [0.0, 0.0, 240.0, 420.0, 540.0, 840.0, 1260.0, 1320.0, 1440.0, 1500.0, 1680.0, 1800.0, 1980.0, 1980.0]]
['depart Ashby', 'arrive Hayward', 1980.0, ['depart Ashby', 'orange Ashby', 'orange MacArthur', 'orange 19th Street', 'orange 12th Street', 'orange Lake Merritt', 'orange Fruitvale', 'orange Coliseum', 'orange San Leandro', 'orange Bay Fair', 'orange Hayward', 'arrive Hayward'], [0.0, 0.0, 240.0, 420.0, 540.0, 720.0, 1020.0, 1260.0, 1500.0, 1740.0, 1980.0, 1980.0]]
['depart Ashby', 'arrive Lafayette', 1139.0, ['depart Ashby', 'red Ashby', 'red MacArthur', 'yellow MacArthur', 'yellow Rockridge', 'yellow Orinda', 'yellow Lafayette', 'arrive Lafayette'], [0.0, 0.0, 

['depart Downtown Berkeley', 'red El Cerrito del Norte', 480.0, ['depart Downtown Berkeley', 'red Downtown Berkeley', 'red North Berkeley', 'red El Cerrito Plaza', 'red El Cerrito del Norte'], [0.0, 0.0, 120.0, 300.0, 480.0]]
['depart Downtown Berkeley', 'red Embarcadero', 1440.0, ['depart Downtown Berkeley', 'red Downtown Berkeley', 'red Ashby', 'red MacArthur', 'red 19th Street', 'red 12th Street', 'red West Oakland', 'red Embarcadero'], [0.0, 0.0, 180.0, 420.0, 600.0, 720.0, 1020.0, 1440.0]]
['depart Downtown Berkeley', 'red Glen Park', 2160.0, ['depart Downtown Berkeley', 'red Downtown Berkeley', 'red Ashby', 'red MacArthur', 'red 19th Street', 'red 12th Street', 'red West Oakland', 'red Embarcadero', 'red Montgomery Street', 'red Powell Street', 'red Civic Center', 'red 16th Street Mission', 'red 24th Street Mission', 'red Glen Park'], [0.0, 0.0, 180.0, 420.0, 600.0, 720.0, 1020.0, 1440.0, 1500.0, 1620.0, 1680.0, 1860.0, 1980.0, 2160.0]]
['depart Downtown Berkeley', 'red MacArthur

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

